In [7]:
import torch
import torch.nn as nn
import torch.onnx
import coremltools as ct
import torch.nn.functional as F

# Define a simple model with grid sampling
class GridSampleModel(nn.Module):
    def __init__(self):
        super(GridSampleModel, self).__init__()
        self.grid_sample_layer = nn.functional.grid_sample

    def forward(self, x, grid):
        return self.grid_sample_layer(x, grid)

# Instantiate the model
input_data = torch.randn(1, 1, 64, 64)  # Adjust dimensions as needed

# Initialize a 2x3 affine matrix
theta = torch.zeros(1, 2, 3)
theta[:, 0, 0] = 1.0  # scale_x
theta[:, 1, 1] = 1.0  # scale_y

# Create the affine grid
grid = F.affine_grid(theta, input_data.size())

# Perform a forward pass
output_pytorch = F.grid_sample(input_data, grid)

# Convert the PyTorch model to ONNX
onnx_path = "grid_sample_model.onnx"
torch.onnx.export(model, (input_data, grid), onnx_path, verbose=True)

# Convert the ONNX model to CoreML
mlmodel_path_fp16 = "grid_sample_model_fp16.mlmodel"
mlmodel_fp16 = ct.converters(onnx_path, target_ios="fp16")
mlmodel_fp16.save(mlmodel_path_fp16)

# Convert the ONNX model to CoreML with default precision (FP32)
mlmodel_path_fp32 = "grid_sample_model_fp32.mlmodel"
mlmodel_fp32 = ct.converters(onnx_path)
mlmodel_fp32.save(mlmodel_path_fp32)

# Compare outputs
output_coreml_fp16 = mlmodel_fp16.predict({"input": input_data.numpy(), "grid": grid.numpy()})
output_coreml_fp32 = mlmodel_fp32.predict({"input": input_data.numpy(), "grid": grid.numpy()})

# Calculate the relative change
relative_change = torch.abs(output_coreml_fp16["output"] - output_coreml_fp32["output"]).mean() / torch.abs(output_coreml_fp32["output"]).mean()

print(f"Relative Change: {relative_change.item() * 100:.2f}%")


Exported graph: graph(%x : Float(1, 1, 64, 64, strides=[4096, 4096, 64, 1], requires_grad=0, device=cpu),
      %grid : Float(1, 64, 64, 2, strides=[8192, 128, 2, 1], requires_grad=0, device=cpu)):
  %2 : Float(1, 1, 64, 64, strides=[4096, 4096, 64, 1], requires_grad=0, device=cpu) = onnx::GridSample[align_corners=0, mode="bilinear", padding_mode="zeros", onnx_name="/GridSample"](%x, %grid), scope: __main__.GridSampleModel:: # /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/nn/functional.py:4304:0
  return (%2)



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/nn/functional.py:4358: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/nn/functional.py:4296: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


TypeError: 'module' object is not callable